In [3]:
import json
import pandas as pd
from time import time
from connections import get_mongo_client
from my_utils import insert_into_mssql, load_my_columns, \
                                        get_numbers_2, map_dff_to_my_columns, \
                                        max_number_find


In [4]:
write_to_table = 'bronze.katm_077_sysinfo'
columns_file = 'katm_077_sysinfo_fields.txt'
columns = load_my_columns(columns_file)
max_number = max_number_find(write_to_table)

In [5]:
max_number

1519666

In [6]:
client = client = get_mongo_client()
db = client['task']
task_collection = db['task']
query = {
    'data.katm_077.return.data.sysinfo': {'$exists': True}
    # ,'number': {'$eq': 7090166}
    ,'number': {'$gt': max_number}
    # ,'number': {'$gt': 1243100, '$lt': 1243200}
    # ,'number': {'$in': numbers}
    
}
projection = {
    'number': 1,
    'data.katm_077.return.data.sysinfo': 1
}

In [7]:
docs = task_collection.find_one(query, projection)

In [8]:
docs

{'_id': ObjectId('65793ff9d12c6f0ca828269b'),
 'number': 1519667,
 'data': {'katm_077': {'return': {'data': {'sysinfo': {'notes': '1.\tПредоставление и использование кредитной информации регулируется законом № ЗРУ-301 «Об обмене кредитной информацией» от 04.10.2011 г.\n2.\tКредитное бюро не проставляет запрета на кредитование, не вносит субъекта кредитной информации в «чёрный» список и не осуществляет никаких корректировок, исправлений, удалений в кредитную информацию, отражённой в кредитном отчёте. Внести изменения вправе, организация, предоставившая информацию, и только в случае, если данные переданы неверно или ошибочно.\n3.\tЕсли субъект кредитной информации не согласен с какой-либо информацией, содержащейся в кредитном отчёте, субъекту кредитной информации необходимо обратиться сперва к поставщику информации, а затем в кредитное бюро с заявлением об оспаривании информации.\n4.\tСкоринговый балл имеет рекомендательный характер и может быть основанием для выдачи или отказа кредитов 

In [3]:
columns

['_id',
 'number',
 'notes',
 'subject_type',
 'org',
 'claim_id',
 'report_type',
 'branch',
 'demand_id',
 'user_id',
 'consent_date',
 'report_name',
 'org_name',
 'demand_date_time',
 'consent_id',
 'claim_date']

In [16]:
client = client = get_mongo_client()
db = client['task']
task_collection = db['task']
query = {
    'data.katm_077.return.data.sysinfo': {'$exists': True}
    # ,'number': {'$eq': 7090166}
    # ,'number': {'$gt': max_num}
    ,'number': {'$gt': 1243100, '$lt': 1243200}
    # ,'number': {'$in': numbers}
    
}
projection = {
    'number': 1,
    'data.katm_077.return.data.sysinfo': 1
}

In [17]:
docs = task_collection.find(query, projection)#.sort('number', -1)

In [10]:
for doc in docs:
    rows = []
    _id = str(doc.get('_id')) 
    number = doc.get('number')
    fields = doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('sysinfo', {})
    if isinstance(fields, dict):
        fields = [fields]
    elif fields is None:
        fields = []
    for field in fields:
        row = {'_id': _id, 'number': number, **field}
        rows.append(row)

    df = pd.DataFrame(rows)
    final_df = map_dff_to_my_columns(df, columns)
    insert_into_mssql_2(final_df, write_to_table)

Error inserting row 0: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]String or binary data would be truncated in table 'RISKDB.BRONZE.katm_077_sysinfo', column 'notes'. Truncated value: '1.\tПредоставление и использование кредитной информации регулируется законом № ЗРУ-301 «Об обмене кре'. (2628) (SQLExecDirectW)")
Error inserting row 0: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]String or binary data would be truncated in table 'RISKDB.BRONZE.katm_077_sysinfo', column 'notes'. Truncated value: '1.\tПредоставление и использование кредитной информации регулируется законом № ЗРУ-301 «Об обмене кре'. (2628) (SQLExecDirectW)")
Error inserting row 0: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]String or binary data would be truncated in table 'RISKDB.BRONZE.katm_077_sysinfo', column 'notes'. Truncated value: '1.\tПредоставление и использование кредитной информации регулируется законом № ЗРУ-301 «Об обмене 